<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch08/section_71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## データセットの読み込み


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dev_src = "/content/drive/MyDrive/SST-2/dev.tsv"
train_src = "/content/drive/MyDrive/SST-2/train.tsv"

In [6]:
%run /content/drive/MyDrive/Colab\ Notebooks/section_70.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 73.7 MB/s eta 0:00:00
語彙数: 3000001
埋め込み次元数 300
(3000000, 300)
3.3527612686157227


/tmp/ipython-input-869294723.py:4: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:206.)
  E[i] = torch.from_numpy(wv[token]) # from_numpyはnumpy配列をPyTorchテンソル型に変換する関数（torch.Tensor）


=== Embedding matrix info ===
E shape: torch.Size([3000001, 300])
Embedding dim (d): 300
Vocab size (V): 3000001
idx2token[1]:</s>


In [9]:
import pandas as pd

In [26]:
train_df = pd.read_csv(train_src, sep='\t')
dev_df = pd.read_csv(dev_src, sep='\t')

In [30]:
def filter_token(row):
  tokens = str(row['sentence']).split() # sentenceが欠損している場合を考えてstrで文字列変換し、NaNを空文字にしている
  input_ids = []
  not_found_count = 0
  for token in tokens:
    if token in token2id:
      input_ids.append(token2id[token])
  return input_ids

### torch.tensorとは
- PyTorchで計算するためのデータ型
  - GPUに載せられる
  - 自動微分（勾配計算）ができる
  - PyTorchのニューラルネットでそのまま使える
- PyTorchのモデルに渡すときは必ずtensorである必要がある

### dtype = torch.longとは
- 64bitの整数型
- int64と同じ
- int32との違いは表現の幅
  - int32は32bitの整数で（約-21億~+21億）
  - int64は64bitの整数で（約-9e18~+9e18）
- embeddingの入力は`torch.long`（int64）必須というルールがある

### labelがfloatな理由
- SST-2は二値分類
- 出力 : 確率（0.0 ~ 1.0）
- 損失関数 : PyTorchのBCELoss や BCEWithLogitsLossは正解ラベルを確率値として扱うのでlabelをfloat型で表現する必要がある。

### torch.tensorの引数
```
torch.tensor(data, dtype=..., device=...)
```
- 今回の場合
  - `data` -> python list
  - `dtype` -> 中身の型
  - `device` -> CPU/GPU
    - PyTorchのtensorはどこに置くかを選べる。
      - CPU : 普通の計算（遅め）
      - GPU : 並列計算が得意（ニューラルネットが速い）
    - modelとdataは同じdeviceに置かないといけない
- tensorに変換することによって後に読み込むときにPyTorchで使用できるようにしている

In [31]:
def row_to_ids(row):
  input_ids = filter_token(row)

  if len(input_ids) == 0:
    return None

  return {
      "text": row["sentence"],
      "label": torch.tensor([row["label"]], dtype=torch.float32),
      "input_ids": torch.tensor(input_ids, dtype=torch.long)
  }

### dropna()とは？
- 欠損値（NaN）を含む行を削除するPandasのメソッド
- 上の関数でlen(input_ids) == 0の場合はNoneを返しているのでNaNで保存される

In [41]:
train_data = train_df.apply(row_to_ids, axis=1).dropna()
dev_data = dev_df.apply(row_to_ids, axis=1).dropna()

In [46]:
# 先頭10件のみを表示
print(train_data.head(10))
print(dev_data.head(10))

0    {'text': 'hide new secretions from the parenta...
1    {'text': 'contains no wit , only labored gags ...
2    {'text': 'that loves its characters and commun...
3    {'text': 'remains utterly satisfied to remain ...
4    {'text': 'on the worst revenge-of-the-nerds cl...
5    {'text': 'that 's far too tragic to merit such...
6    {'text': 'demonstrates that the director of su...
7    {'text': 'of saucy ', 'label': [tensor(1.)], '...
8    {'text': 'a depressed fifteen-year-old 's suic...
9    {'text': 'are more deeply thought through than...
dtype: object
0    {'text': 'it 's a charming and often affecting...
1    {'text': 'unflinchingly bleak and desperate ',...
2    {'text': 'allows us to hope that nolan is pois...
3    {'text': 'the acting , costumes , music , cine...
4    {'text': 'it 's slow -- very , very slow . ', ...
5    {'text': 'although laced with humor and a few ...
6    {'text': 'a sometimes tedious film . ', 'label...
7    {'text': 'or doing last year 's taxes with you

In [47]:
# 件数が減っているかを確認する
print("train:", len(train_df), "->", len(train_data))
print("dev  :", len(dev_df),   "->", len(dev_data))

train: 67349 -> 66650
dev  : 872 -> 872


### ilocとは
- index番号でデータを取り出すpandasの仕組み
- iloc[0]だと先頭行のデータを取得する

In [49]:
# 一件目のデータを詳しく表示する
first_data = train_data.iloc[0]
print(first_data["text"])
print(first_data["label"])
print(first_data["input_ids"])

hide new secretions from the parental units 
tensor([0.])
tensor([  5785,     66, 113845,     18,     12,  15095,   1594])
